## Importações e carregamentos de datasets

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import re

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Arquivos de aula/TCC/Datasets/ABSAPT 2022 - Training.csv', sep=";", index_col='id')
df_train

,review,polarity,aspect,start_position,end_position
id,,,,,
0,O hotel é perto de todos os pontos principais ...,0,quarto,152,158
1,Viajamos eu e minha irmã. O hotel tem uma extr...,1,elevador,63,71
2,Estive por 8 dias hospedado neste hotel com mi...,1,café da manhã,209,222
3,Adorei a estadia. Porto Alegre foi sensacional...,-1,hotel,378,383
4,"O hotel tem ótima localização, fizemos vários ...",1,internet,216,224
...,...,...,...,...,...
3106,Reservei esse hotel através do pacote da TAM V...,-1,recepção,588,596
3107,Fomos muito bem atendido na chegada disponibil...,1,quarto,128,134
3108,"Excelente hotel, sem cassino e aquela confusao...",0,shopping,99,107


In [ ]:
df_train['polarity'].value_counts()

 1    2112
-1     527
 0     472
Name: polarity, dtype: int64

In [ ]:
df_domain_words = pd.read_csv('/content/drive/MyDrive/Arquivos de aula/TCC/Datasets/training_domain_words.csv', index_col=0)
df_domain_words

,word,relation,related_word
0,ababelado,-1,ajeitado
2,ababelado,-1,arrumado
4,ababelado,-1,organizado
12,ababelado,1,confuso
14,ababelado,1,desarrumado
...,...,...,...
64327,íntegro,1,pleno
64333,íntimo,1,particular
64334,íntimo,1,pessoal
64340,ínubo,1,solteiro


## Criação do dataset de sinônimos e antônimos do domínio de treino

In [ ]:
# !wget https://raw.githubusercontent.com/stavarengo/portuguese-brazilian-synonyms/master/layout-one.txt
!wget https://raw.githubusercontent.com/stavarengo/portuguese-brazilian-synonyms/master/layout-two.txt

--2023-08-30 18:05:20--  https://raw.githubusercontent.com/stavarengo/portuguese-brazilian-synonyms/master/layout-two.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10439527 (10.0M) [text/plain]
Saving to: ‘layout-two.txt’

layout-two.txt      100%[===================>]   9.96M  --.-KB/s    in 0.1s    

2023-08-30 18:05:20 (71.0 MB/s) - ‘layout-two.txt’ saved [10439527/10439527]



In [ ]:
df_syns = pd.read_csv('/content/layout-two.txt', sep=' ', names=['category', 'word', 'relation', 'related_word'], on_bad_lines='skip')
df_syns

,category,word,relation,related_word
0,[NOUN],conhecimento,SYNONYMOUS_OF,escola
1,[ADJECTIVE],indolente,SYNONYMOUS_OF,omisso
2,[NOUN],canhambola,SYNONYMOUS_OF,quilombola
3,[ADJECTIVE],constante,SYNONYMOUS_OF,freqüente
4,[VERB],pretender,SYNONYMOUS_OF,visar
...,...,...,...,...
251383,[VERB],engordar,ANTONYM_OF,acanavear
251384,[ADJECTIVE],coxo,SYNONYMOUS_OF,pencó
251385,[VERB],peraltear,SYNONYMOUS_OF,peralvilhar
251386,[NOUN],desatino,ANTONYM_OF,lucidez


In [ ]:
df_syns.drop(df_syns[df_syns.category != '[ADJECTIVE]'].index, inplace=True)

In [ ]:
df_syns.drop(columns=['category'], inplace=True)

In [ ]:
df_syns.sort_values(by= ['word','relation','related_word'], inplace=True)

In [ ]:
df_syns.reset_index(drop=True, inplace=True)

In [ ]:
df_syns.loc[df_syns.relation == 'SYNONYMOUS_OF', 'relation'] = 1
df_syns.loc[df_syns.relation == 'ANTONYM_OF', 'relation'] = -1
df_syns

In [ ]:
df_syns.shape

In [ ]:
df_syns.to_csv('/content/drive/MyDrive/Arquivos de aula/TCC/Datasets/synonyms_antonyms.csv')

In [ ]:
df_syns = pd.read_csv('/content/drive/MyDrive/Arquivos de aula/TCC/Datasets/synonyms_antonyms.csv', index_col='Unnamed: 0')
df_syns.head()

In [ ]:
len(set(df_syns['related_word'].values.tolist()))

In [ ]:
unique_related_words = set(df_syns['related_word'].values.tolist())

In [ ]:
import re

final_string = ''
for text in df_train['review'].values:
  final_string += re.sub('[^a-zA-Z\s]+', '', text) + ' '
final_string = final_string.replace('\n\n', ' ').lower()

In [ ]:
unique_reviews = set(final_string.split())

In [ ]:
unique_words = unique_related_words & unique_reviews

In [ ]:
df_domain_words = df_syns[df_syns['related_word'].isin(unique_words)]
df_domain_words

In [ ]:
df_domain_words.to_csv('/content/drive/MyDrive/Arquivos de aula/TCC/Datasets/training_domain_words.csv')

## Geração de dados com SpaCy

### Configuração do matcher

In [ ]:
!pip -qqq install spacy
!python -m spacy download pt_core_news_lg --quiet

2023-08-27 19:12:14.039765: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-27 19:12:17.121435: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-27 19:12:18.874624: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-27 19:12:18.875100: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [ ]:
import spacy
from spacy.matcher import Matcher
from spacy.matcher import DependencyMatcher

nlp = spacy.load("pt_core_news_lg")

In [ ]:
doc = nlp("O quarto do hotel é bonito e cheiroso.")
spacy.displacy.render(doc, style='dep', jupyter=True)

In [ ]:
pattern = [
  {
    "RIGHT_ID": "target",
    "RIGHT_ATTRS": {"POS": "NOUN"}
  },

  {
    "LEFT_ID": "target",
    "REL_OP": ">>",
    "RIGHT_ID": "modifier",
    "RIGHT_ATTRS": {"POS": "ADJ"}
  },
]

matcher = DependencyMatcher(nlp.vocab)
matcher.add("FOUNDED", [pattern])

### Geração dos dados

In [ ]:
augmented_list = []
for index, review in enumerate(df_train.values):
  doc = nlp(review[0])

  for match_id, (target, modifier) in matcher(doc):
    if review[2] in doc[target].orth_.lower():
      curr_word = doc[modifier].orth_
      related_words_temp_list = df_domain_words[(df_domain_words['word'] == curr_word) & (df_domain_words['relation'] == -1)].drop(columns= ['word']).values
      for related_word in related_words_temp_list:
        temp_row = review.copy()
        temp_row[0] = temp_row[0].replace(curr_word, related_word[1])
        temp_row[1] = -temp_row[1] if related_word[0] == -1 else temp_row[1]
        augmented_list.append(temp_row)

  # for word in set(df_domain_words['word'].values):
  #   word_index = review[0].find(' ' + word + ' ')
  #   if word_index != -1:
  #     synonyms_temp_list = df_domain_words[(df_domain_words['word'] == word) & (df_domain_words['relation'] == 1)]['related_word'].values
  #     for synonym in synonyms_temp_list:
  #       temp_row = review.copy()
  #       temp_row[0] = temp_row[0].replace(word, synonym)
  #       augmented_list.append(temp_row)

In [ ]:
df_train.sort_values(['review','polarity'])

In [ ]:
df_augmented = pd.DataFrame(augmented_list, columns=df_train.columns)
df_augmented.sort_values(['review', 'polarity'])

In [ ]:
df_augmented.shape

In [ ]:
# df_augmented.to_csv('/content/drive/MyDrive/Arquivos de aula/TCC/Datasets/synonyms_training_augmented.csv')

In [ ]:
df_testing = pd.read_csv('/content/drive/MyDrive/Arquivos de aula/TCC/Datasets/antonyms_training_augmented.csv')
df_merged = df_augmented.merge(df_testing,how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
df_merged.reset_index(drop=True, inplace=True)
df_merged.sort_values(['review','polarity'])

In [ ]:
print(df_merged['review'].values[233] + '\n' + df_train['review'].values[2263])
df_merged['review'].values[233] == df_train['review'].values[2263]

In [ ]:
aspects_polarity = pd.DataFrame({"Count": df_train["polarity"].value_counts()})
aspects_polarity_augmented = pd.DataFrame({"Count": df_augmented["polarity"].value_counts()})

plt.subplot(1,2,1)
sns.barplot(x=["Positivo", "Neutro", "Negativo"], y=aspects_polarity["Count"], orient='v')
plt.title("Aspect polarities")
plt.ylim(0, 2500)

plt.subplot(1,2,2)
sns.barplot(x=["Positivo", "Neutro", "Negativo"], y=aspects_polarity_augmented["Count"], orient='v')
plt.title("Aspect polarities augmented")
plt.ylim(0, 2500)

plt.subplots_adjust(wspace=0.4)
plt.show()

In [ ]:
aspects = pd.DataFrame({"Count": df_train["aspect"].value_counts()})
aspects_augmented = pd.DataFrame({"Count": df_augmented["aspect"].value_counts()})

plt.subplot(2,1,1)
sns.barplot(y=aspects[0:10].index, x=aspects[0:10]["Count"], orient='h')
plt.title("Top 10 aspects")
plt.xlim(0, 600)

plt.subplot(2,1,2)
sns.barplot(y=aspects[0:10].index, x=aspects_augmented[0:10]["Count"], orient='h')
plt.title("Top 10 aspects augmented")
plt.xlim(0, 600)

plt.subplots_adjust(hspace=0.4)
plt.show()

In [ ]:
from spacy import displacy

review_n = 3107

doc = nlp(df_train['review'].values[review_n])

# for sentence in doc.sents:
#         if df_train['aspect'].values[review_n] in str(sentence).lower():
#           for token in sentence:
#             if token.pos_ == "ADJ":
#               aspect_sentece = sentence
#               displacy.render(aspect_sentece, style='dep', jupyter=True)
#               break
aspect_sentence = ''
for sentence in doc.sents:
        if df_train['aspect'].values[review_n] in str(sentence).lower():
              aspect_sentence = sentence
              displacy.render(aspect_sentence, style='dep', jupyter=True)

print("{:25} {:10} {:10} {:20} {:10} {:10} {:50} {:50}".format('Token','Pos-Tag','Dep','Head','Pos-Head','NER', 'Lefts', 'Rights'))
print("_" * 150)
for token in aspect_sentence:
    print("{:25} {:10} {:10} {:20} {:10} {:10} {:50} {:50}".format(token.text, token.pos_ , token.dep_, token.head.text, token.head.pos_, token.ent_type_,  "-".join([w.text for w in token.lefts]), "-".join([w.text for w in token.rights])))

## Geração de dados com Transformers

In [ ]:
!pip -qqq install transformers
from transformers import pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.7 MB/s eta 0:00:00


###Albertina unmasker

In [ ]:
unmasker = pipeline('fill-mask', model='PORTULAN/albertina-ptbr')

In [ ]:
unmasker('Hotel muito confortável, antigo mas muito bem conservado. Ótimos quartos, roupa de cama [MASK] e café da manhã excelente. Ótima localização (com vista para a Torre Eiffel) e bem pertinho dela. Um dos melhores em que já me hospedei, principalmente se considerarmos as tarifas, bastante razoáveis quando comparadas a outras nem sempre tão adequadas.')

[{'score': 0.8573883771896362,
  'token': 74763,
  'token_str': 'boa',
  'sequence': 'Hotel muito confortável, antigo mas muito bem conservado. Ótimos quartos, roupa de cama boa e café da manhã excelente. Ótima localização (com vista para a Torre Eiffel) e bem pertinho dela. Um dos melhores em que já me hospedei, principalmente se considerarmos as tarifas, bastante razoáveis quando comparadas a outras nem sempre tão adequadas.'},
 {'score': 0.061461567878723145,
  'token': 101,
  'token_str': ',',
  'sequence': 'Hotel muito confortável, antigo mas muito bem conservado. Ótimos quartos, roupa de cama, e café da manhã excelente. Ótima localização (com vista para a Torre Eiffel) e bem pertinho dela. Um dos melhores em que já me hospedei, principalmente se considerarmos as tarifas, bastante razoáveis quando comparadas a outras nem sempre tão adequadas.'},
 {'score': 0.019162097945809364,
  'token': 106767,
  'token_str': 'linda',
  'sequence': 'Hotel muito confortável, antigo mas muito bem 

In [ ]:
list_temp = pd.read_csv('/content/drive/MyDrive/Arquivos de aula/TCC/Datasets/synonyms_training__transformers_augmented_1723.csv', index_col = 0)
list_temp = list_temp.values.tolist()

In [ ]:
# augmented_list = []
# for index, review in enumerate(enumerate(df_train.values[1722:],1722)):
augmented_list = list_temp
for index, review in enumerate(df_train.values[1723:],1723):
  doc = nlp(review[0])
  for match_id, (target, modifier) in matcher(doc):
    if len(augmented_list) % 100 == 0:
      df_augmented = pd.DataFrame(augmented_list, columns=df_train.columns)
      df_augmented.to_csv('/content/drive/MyDrive/Arquivos de aula/TCC/Datasets/synonyms_training__transformers_augmented_'+str(index)+'.csv')

    if review[2] in doc[target].orth_.lower():
      curr_word = doc[modifier].orth_
      if review[0].count(curr_word) > 1:
        splited = review[0].split()
        indexes = np.array([(word == curr_word) * index for index, word in enumerate(splited)])
        closest_index = (np.abs(indexes - review[3])).argmin()
        splited[closest_index] = '[MASK]'
        ' '.join(splited)
      else:
        temp_review = review[0].replace(curr_word, '[MASK]')
      predictions = unmasker(temp_review)
      predictions = [(prediction['token_str'], prediction['sequence']) for prediction in predictions]
      for word, sentence in predictions:
        if word not in df_domain_words['related_word'].values or word == curr_word:
          continue
        temp_row = review.copy()
        temp_row[0] = sentence
        augmented_list.append(temp_row)

df_augmented.to_csv('/content/drive/MyDrive/Arquivos de aula/TCC/Datasets/synonyms_training__transformers_augmented.csv')

In [ ]:
df_transformers = pd.read_csv('/content/drive/MyDrive/Arquivos de aula/TCC/Datasets/synonyms_training__transformers_augmented.csv', index_col = 0)
df_transformers

,review,polarity,aspect,start_position,end_position
0,O hotel é perto de todos os pontos principais ...,0,quarto,152,158
1,O hotel é perto de todos os pontos principais ...,0,quarto,152,158
2,O hotel é perto de todos os pontos principais ...,0,quarto,152,158
3,Viajamos eu e minha irmã. O hotel tem uma extr...,1,elevador,63,71
4,Viajamos eu e minha irmã. O hotel tem uma extr...,1,elevador,63,71
...,...,...,...,...,...
2195,a localização achei muito boa... bem localizad...,1,funcionários,256,268
2196,"Venho de chegar de minha viagem de Paris, Pass...",1,hotel,75,80
2197,"Proximo ao Metro (Republique), onde há conexõe...",1,cama,248,252
2198,"Proximo ao Metro (Republique), onde há conexõe...",1,cama,248,252


###Pos tagger

In [ ]:
pos_tagger = pipeline(model='lisaterumi/postagger-portuguese',tokenizer='lisaterumi/postagger-portuguese')

In [ ]:
df_train.values[1]

array(['Viajamos eu e minha irmã. O hotel tem uma extrutura excelente! Elevadores modernos e rápidos. O quarto bem climatizado e roupas de cama novas e limpas. O café da manhã farto e variado. O fato de ser ao lado da gare Montparnasse facilitou a ida a Versailles, pois pudemos pegar um trem lá. O ponto negativo é que o metrô não é muito perto, (6 quadras grandes) e a caminhada no frio muito angustiante! Mas retornaria com certeza!',
       1, 'elevador', 63, 71], dtype=object)

In [ ]:
pos_tagger(df_train.values[1][0])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'V',
  'score': 0.9996333,
  'index': 1,
  'word': 'Via',
  'start': 0,
  'end': 3},
 {'entity': 'V',
  'score': 0.9997223,
  'index': 2,
  'word': '##jam',
  'start': 3,
  'end': 6},
 {'entity': 'V',
  'score': 0.99972934,
  'index': 3,
  'word': '##os',
  'start': 6,
  'end': 8},
 {'entity': 'PROPESS',
  'score': 0.9983615,
  'index': 4,
  'word': 'eu',
  'start': 9,
  'end': 11},
 {'entity': 'KC',
  'score': 0.9993303,
  'index': 5,
  'word': 'e',
  'start': 12,
  'end': 13},
 {'entity': 'PROADJ',
  'score': 0.99843496,
  'index': 6,
  'word': 'minha',
  'start': 14,
  'end': 19},
 {'entity': 'N',
  'score': 0.9995415,
  'index': 7,
  'word': 'irmã',
  'start': 20,
  'end': 24},
 {'entity': 'PU',
  'score': 0.9998746,
  'index': 8,
  'word': '.',
  'start': 24,
  'end': 25},
 {'entity': 'ART',
  'score': 0.99958557,
  'index': 9,
  'word': 'O',
  'start': 26,
  'end': 27},
 {'entity': 'N',
  'score': 0.99969935,
  'index': 10,
  'word': 'hotel',
  'start': 28,
  'end': 3

In [ ]:
def removeTags(index, word_list, word = ''):
  curr_word = (word_list[index]['word']).replace('#','')+word
  if '#' not in word_list[index]['word']:
    return curr_word
  else:
    output = removeTags(index-1, word_list, curr_word)

  return output

In [ ]:
def tagging(phrase):
  tagged_review = pos_tagger(phrase)
  is_marked = False
  word_list = []
  for index, row in enumerate(tagged_review):
    if index < (len(tagged_review)-1) and '#' in tagged_review[index+1]['word']:
      is_marked = True
      continue
    elif is_marked:
      word = removeTags(index, tagged_review)
      is_marked = False
    else:
      word = row['word']

    word_list.append((word,row['entity']))
  return word_list

In [ ]:
def augmentData(augmented_list, review, word):
  related_words_temp_list = df_domain_words[(df_domain_words['word'] == word) & (df_domain_words['relation'] == -1)].drop(columns= ['word']).values
  for related_word in related_words_temp_list:
    temp_row = review.copy()
    temp_row[0] = temp_row[0].replace(word, related_word[1])
    temp_row[1] = -temp_row[1] if related_word[0] == -1 else temp_row[1]
    augmented_list.append(temp_row)

In [ ]:
augmented_list = []
for review in df_train.values:
  aspect_len = len(review[2].split())
  word_list = tagging(review[0])

  for index, (word, tag) in enumerate(word_list):
    if aspect_len > 1 and index >= aspect_len-1:
      curr_word = ''
      for aux in range(index-(aspect_len-1),index+1):
        curr_word += ' ' + word_list[aux][0]
    else:
      curr_word = word

    if review[2] in curr_word.lower():
      min_range = index-3 if index > 2 else 0
      max_range = index+4 if len(word_list) > index+3 else len(word_list)
      for w in range(min_range, max_range):
        if word_list[w][1] == 'ADJ':
          augmentData(augmented_list, review, word_list[w][0])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
df_augmented = pd.DataFrame(augmented_list, columns=df_train.columns)
df_augmented

,review,polarity,aspect,start_position,end_position
0,"Quarto muito pequeno, porém a cama é confortáv...",0,limpeza,104,111
1,O Distrik é um ótimo hotel. Equipe atenciosa n...,-1,quarto,66,72
2,O Distrik é um ótimo hotel. Equipe atenciosa n...,-1,quarto,66,72
3,O Distrik é um ótimo hotel. Equipe atenciosa n...,-1,quarto,66,72
4,O Distrik é um ótimo hotel. Equipe atenciosa n...,-1,quarto,66,72
...,...,...,...,...,...
1102,"Ótima localização, perto de vários pontos turí...",-1,café da manhã,403,416
1103,O hotel é charmoso e aconchegante. Tem um ótim...,-1,café da manhã,48,61
1104,O hotel tem valor acessível levando-se em cont...,1,café da manhã,232,245
1105,O hotel tem valor acessível levando-se em cont...,1,café da manhã,232,245


In [ ]:
df_augmented.to_csv('/content/drive/MyDrive/Arquivos de aula/TCC/Datasets/synonyms_training__transformers_pos_tagger_augmented.csv')